# Outline
Text goes here

In [23]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from citipy import citipy
import random
import requests
import json
import io



# Get API key from api_keys.py
import api_keys
api_ow = api_keys.api_ow

In [24]:
# Sample API call for OpenWeather geolocation:
# http://api.openweathermap.org/geo/1.0/direct?q=CITYNAME,STATECODE,COUNTRYCODE&limit=LIMIT&appid=APIKEY

# Sample API call for OpenWeather weather data:
# units = "imperal"
#api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + LATITUDE + "&lon=" + LONGITUDE + "&appid=" + api_ow + "&units=" + units

In [65]:
# Functions

# Generates a random number between -180 and 180 degrees (0 is Greenwich, England).
def genLatitude():
    randomLat = random.uniform(-180, 180)
    return round(randomLat, 5)

# Generates a random number between -90 and 90 degrees (0 is equator)
def genLongitude():
    randomLong = random.uniform(-90, 90)
    return round(randomLong, 5)

# Send an OpenWeather API request with a city name, a country code, and a limit of the number of responses. In most cases the limit should be 1.
# Return the details of the city in a single-row dataframe.
def getCityCoordinates(city, country, limit):
    api_string = "http://api.openweathermap.org/geo/1.0/direct?q=" + city + "," + country + "&limit=" + str(limit) + "&appid=" + api_ow + "&lang=en"
    result = requests.get(api_string)
    print("Sending API request to geolocate city: " + api_string)
    print("Got API response: " + result.content.decode("utf-8"))
    df_result = pd.read_json(result.content.decode("utf-8"))
    result_columns = ["name", "lat", "lon", "country", "state"]
    df_result = df_result.loc[:, result_columns]
    return df_result

# Send an OpenWeather API request with a set of latitude and longitude coordinates and the type of units (valid values are "standard", "metric", "imperial").
# Response will be in JSON. Extract temperature, humidity, cloudiness, wind speed.
# Return a single-row dataframe.
def getWeatherData(lat, long, units):
    api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + str(lat) + "&lon=" + str(long) + "&appid=" + api_ow + "&units=" + units
    result = requests.get(api_string)
    print("Sending API request for weather data: " + api_string)
    print("Got API response: " + result.content.decode("utf-8"))
    json_result = result.content.decode("utf-8")
    json_result = json.loads(json_result)
    return json_result






    

In [ ]:
# scraps

    #json_as_file = io.BytesIO(json_result)
    #dfs = []
    #with open(json_result, 'r') as f:
    #    for line in f:
    #        data = json.loads(line)
    #        df = pd.DataFrame.from_dict(data)
    #        dfs.append(df)
    #df_result = pd.concat(dfs, ignore_index=True)

    #df_result = pd.DataFrame.from_dict(data_json)
    #print(df_result)




    #df_result = pd.read_json(result.content.decode("utf-8"))
    #print(df_result)

In [26]:
# Generate 500 random sets of long/lat coordinates.
# Get the nearest city to each coordinates. If it's not in the list, add its weather data.
cities_500 = []
count = 1
while count <= 500:
    lat = genLatitude()
    long = genLongitude()
    nearest = citipy.nearest_city(lat, long)
    city = [nearest.city_name, nearest.country_code]

    # Only add the city if it's unique
    if city not in cities_500:
        cities_500.append(city)
        count += 1

In [ ]:
# Dataframe to hold city data.
df_cities = pd.DataFrame(columns=["City Name", "Latitude", "Longitude", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind speed (MPH)"])
df_cities

In [75]:
df_test = getCityCoordinates("iqaluit", "ca", 1)
json_test = getWeatherData(df_test["lat"][0], df_test["lon"][0], "imperial")
json_test


Sending API request to geolocate city: http://api.openweathermap.org/geo/1.0/direct?q=iqaluit,ca&limit=1&appid=8d5b3e696eb6fbf16ef9ac306b2f3965&lang=en
Got API response: [{"name":"Iqaluit","local_names":{"lt":"Ikaluitas","he":"איקאלואיט","ru":"Икалуит","fr":"Iqaluit","el":"Ικάλουιτ","iu":"ᐃᖃᓗᐃᑦ","uk":"Ікалуїт","en":"Iqaluit","ar":"إقالويت"},"lat":63.74944,"lon":-68.521857,"country":"CA","state":"Nunavut"}]
Sending API request for weather data: https://api.openweathermap.org/data/2.5/weather?lat=63.74944&lon=-68.521857&appid=8d5b3e696eb6fbf16ef9ac306b2f3965&units=imperial
Got API response: {"coord":{"lon":-68.5219,"lat":63.7494},"weather":[{"id":600,"main":"Snow","description":"light snow","icon":"13d"}],"base":"stations","main":{"temp":32.92,"feels_like":29.77,"temp_min":32.92,"temp_max":33.75,"pressure":1005,"humidity":93},"visibility":10000,"wind":{"speed":3.44,"deg":150},"clouds":{"all":75},"dt":1718142115,"sys":{"type":2,"id":47658,"country":"CA","sunrise":1718086585,"sunset":17181

{'coord': {'lon': -68.5219, 'lat': 63.7494},
 'weather': [{'id': 600,
   'main': 'Snow',
   'description': 'light snow',
   'icon': '13d'}],
 'base': 'stations',
 'main': {'temp': 32.92,
  'feels_like': 29.77,
  'temp_min': 32.92,
  'temp_max': 33.75,
  'pressure': 1005,
  'humidity': 93},
 'visibility': 10000,
 'wind': {'speed': 3.44, 'deg': 150},
 'clouds': {'all': 75},
 'dt': 1718142115,
 'sys': {'type': 2,
  'id': 47658,
  'country': 'CA',
  'sunrise': 1718086585,
  'sunset': 1718160687},
 'timezone': -14400,
 'id': 5983720,
 'name': 'Iqaluit',
 'cod': 200}

In [81]:
df_columns = ["City Name", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind Speed (MPH)"]
df_weather = pd.DataFrame(columns=df_columns)
row_to_add = {
    "City Name": json_test["name"],
    "Temperature (F)": json_test["main"]["temp"],
    "Humidity (%)": json_test["main"]["humidity"],
    "Cloudiness (%)": json_test["clouds"]["all"],
    "Wind Speed (MPH)": json_test["wind"]["speed"]
}

df_weather = df_weather.add(row_to_add)
df_weather


,City Name,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
